In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
import numpy as np
import cv2

In [ ]:
def create_dataset(batch_size, input_image, output_label):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((input_image, output_label))

    # Resize and normalize images in a lambda function
    def process_images(image, label):
        image = tf.image.resize(image, [224, 224], method=tf.image.ResizeMethod.BICUBIC)
        image = (image - 125.0) / 255.0
        return image, label

    # Apply the image processing function to all images
    dataset = dataset.map(process_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

    # Prefetch for performance
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
model=keras.Sequential(
    [
     keras.Input(shape=(224,224,3)),
     layers.Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.Conv2D(filters=512,kernel_size=3,padding="same",activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Flatten(),
     layers.Dense(4096,activation="relu"),
     layers.Dropout(rate=0.2),
     layers.Dense(4096,activation="relu"),
     layers.Dropout(rate=0.2),
     layers.Dense(10)
    ]
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 224, 224, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                        

In [ ]:
batch_size=50
epochs=25
learningrate=0.01
loss_fun=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer=keras.optimizers.SGD(learning_rate=learningrate)
trainable_dataset=create_dataset(batch_size,x_train,y_train)

In [ ]:
train_dataset = create_dataset(batch_size, x_train, y_train)


In [ ]:
@tf.function
def train_step(x, y, model, loss_fun, optimizer, accuracy):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fun(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    # Calculate predictions and update accuracy
    predictions = tf.argmax(logits, axis=1)
    accuracy.update_state(y, predictions)
    return loss_value

accuracy = tf.keras.metrics.Accuracy()
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train, model, loss_fun, optimizer, accuracy)
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))
            print("Training accuracy: %.4f" % float(accuracy.result()))
    # Reset the metrics for the next epoch
    accuracy.reset_states()




Start of epoch 0
Training loss (for one batch) at step 0: 3.7808
Seen so far: 50 samples
Training accuracy: 0.1200
Training loss (for one batch) at step 200: 1.2771
Seen so far: 10050 samples
Training accuracy: 0.3550
Training loss (for one batch) at step 400: 1.5463
Seen so far: 20050 samples
Training accuracy: 0.4049
Training loss (for one batch) at step 600: 1.4062
Seen so far: 30050 samples
Training accuracy: 0.4427
Training loss (for one batch) at step 800: 1.3248
Seen so far: 40050 samples
Training accuracy: 0.4688

Start of epoch 1
Training loss (for one batch) at step 0: 1.0751
Seen so far: 50 samples
Training accuracy: 0.6400
Training loss (for one batch) at step 200: 1.3119
Seen so far: 10050 samples
Training accuracy: 0.6101
Training loss (for one batch) at step 400: 0.9090
Seen so far: 20050 samples
Training accuracy: 0.6218
Training loss (for one batch) at step 600: 0.8647
Seen so far: 30050 samples
Training accuracy: 0.6362
Training loss (for one batch) at step 800: 0.96

KeyboardInterrupt: ignored

In [ ]:
test_dataset = create_dataset(batch_size, x_test, y_test)


In [ ]:
def test_step(x, y, model, loss_fun,  accuracy):
    logits = model(x, training=False)
    loss_value = loss_fun(y, logits)

    # Calculate predictions and update accuracy
    predictions = tf.argmax(logits, axis=1)
    accuracy.update_state(y, predictions)
    return loss_value


accuracy = tf.keras.metrics.Accuracy()
for epoch in range(epochs):
    accuracy_epoch=0
    number_batches=0
    print("\nStart of epoch %d" % (epoch,))
    for step, (x_batch_test, y_batch_test) in enumerate(test_dataset):
        loss_value = test_step(x_batch_test, y_batch_test, model, loss_fun, accuracy)
        if step % 1 == 0:
            # print(
            #     "test loss (for one batch) at step %d: %.4f"
            #     % (step, float(loss_value))
            # )
            # print("Seen so far: %s samples" % ((step + 1) * batch_size))
            accuracy_epoch+= float(accuracy.result())
            number_batches+=1

    # Reset the metrics for the next epoch
    print("acuracy in each epochs",accuracy_epoch/number_batches)
    accuracy.reset_states()





Start of epoch 0
acuracy in each epochs 0.7845233634114266

Start of epoch 1
acuracy in each epochs 0.7833083656430244

Start of epoch 2
acuracy in each epochs 0.7853406476974487

Start of epoch 3
acuracy in each epochs 0.7823028919100762

Start of epoch 4
acuracy in each epochs 0.7843613344430923

Start of epoch 5
acuracy in each epochs 0.7821042340993881

Start of epoch 6
acuracy in each epochs 0.7841131588816643

Start of epoch 7
acuracy in each epochs 0.7822969123721123

Start of epoch 8
acuracy in each epochs 0.7812519335746765

Start of epoch 9
acuracy in each epochs 0.7820750480890274

Start of epoch 10
acuracy in each epochs 0.7806635066866875

Start of epoch 11
acuracy in each epochs 0.7822392645478249

Start of epoch 12
acuracy in each epochs 0.7819165906310082

Start of epoch 13
acuracy in each epochs 0.781324745118618

Start of epoch 14
acuracy in each epochs 0.7841784811019897

Start of epoch 15
acuracy in each epochs 0.782395147383213

Start of epoch 16
acuracy in each e

In [ ]:
test_dataset = create_dataset(batch_size, x_test, y_test)

Difference is batch normalization dropout feature between layers. which help in converging models in less number of interations.
https://github.com/tensorflow/models/tree/master/research/slim (github link)